<a href="https://colab.research.google.com/github/Zia-Ul-Hasan/Churn-Prediction-IBM-telco/blob/main/scratchcode8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np
import joblib
import random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN
from sklearn.linear_model import LogisticRegression


path='/content/Telco_customer_churn.xlsx'
df = pd.read_excel(path)
print (df.info())
#removing unimportant columns from the dataset
df= df.drop(['CustomerID','Country','Count','State','Zip Code','Lat Long','Longitude','Latitude','City', 'Churn Score','Churn Label'], axis=1)
print (df.info())
df.to_csv('required_data_only.csv')

def unique_values_col (df):
 for column in df:
  if df[column].dtypes=='object':
   print(f'{column} : {df[column].unique()}')
 return None


def replace_t(df):
    print(unique_values_col(df))
    for column in df.columns:
        df.loc[df[column] == 'No internet service', column] = 'No'
        df.loc[df[column] == 'No phone service', column] = 'No'
    print(unique_values_col(df))
    return

replace_t(df)
df['Monthly Charges']= df['Monthly Charges'].astype(float)
df['CLTV']= df['CLTV'].astype(float)
df['Total Charges']= pd.to_numeric(df['Total Charges'], errors='coerce')
df['Tenure Months']= pd.to_numeric(df['Tenure Months'], errors='coerce')
df['Churn Value']= pd.to_numeric(df['Churn Value'], errors='coerce')
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [67]:
#removing null values
df= df.dropna(subset=['Total Charges'])
print(df.isnull().sum())


#for removing people who didnt churn to begin with
df2= df.dropna(subset=['Churn Reason'])
print(df2.isnull().sum())


y1= df['Churn Value']

drop_c= ['Churn Value','Churn Reason']
x1 = df.drop(columns=drop_c)

#for finding columns that require label encoding, one hot encoding and numerical columns
num_col=[]
LE_col=[]
OE_col=[]
for col in x1:
  if pd.api.types.is_numeric_dtype(x1[col]):
    num_col.append(col)
  else:
   if x1[col].nunique() == 2:
    LE_col.append(col)
   elif x1[col].nunique()>2:
    OE_col.append(col)
print (f' the column to be label encoded:{LE_col}')
print (f' the column to be One Hot Endoded encoded:{OE_col}')
print(f'the numerical columns are: {num_col}')

#dummy/ one hot encoding

def one_hot_encode_columns(df, columns):
    for column in columns:
        if column in df.columns:
            one_hot = pd.get_dummies(df[column], prefix=column)
            df = df.drop(column, axis=1)
            df = pd.concat([df, one_hot], axis=1)
        else:
            print(f"Column '{column}' not found in DataFrame.")
    return df



#Label Encoding
LE = LabelEncoder()
def Label_encoding(tt):
 for col in LE_col:
  tt[col]=LE.fit_transform(tt[col])
 return tt


Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Total Charges           0
Churn Value             0
CLTV                    0
Churn Reason         5163
dtype: int64
Gender               0
Senior Citizen       0
Partner              0
Dependents           0
Tenure Months        0
Phone Service        0
Multiple Lines       0
Internet Service     0
Online Security      0
Online Backup        0
Device Protection    0
Tech Support         0
Streaming TV         0
Streaming Movies     0
Contract             0
Paperless Billing    0
Payment Method       0
Monthly Charges      0
T

In [73]:
#splitting the dataset

x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=42)
x1_train=one_hot_encode_columns(x1_train, OE_col)
x1_test=one_hot_encode_columns(x1_test, OE_col)
x1_train=Label_encoding(x1_train)
x1_test=Label_encoding(x1_test)

print(x1_train.dtypes)



# applying SMOTKE
smote_tomek = SMOTETomek(random_state=42)
x1_resampled, y1_resampled = smote_tomek.fit_resample(x1_train, y1_train)

# Train Random Forest
rf = RandomForestClassifier(random_state=42,n_estimators=500, max_depth=20, criterion='entropy', min_samples_split=10, min_samples_leaf=4,max_features='sqrt', bootstrap=False, class_weight='balanced')
rf.fit(x1_resampled, y1_resampled)

# Predict on the test set
y1_pred = rf.predict(x1_test)

# Evaluate the model
print(confusion_matrix(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

Gender                                        int64
Senior Citizen                                int64
Partner                                       int64
Dependents                                    int64
Tenure Months                                 int64
Phone Service                                 int64
Multiple Lines                                int64
Online Security                               int64
Online Backup                                 int64
Device Protection                             int64
Tech Support                                  int64
Streaming TV                                  int64
Streaming Movies                              int64
Paperless Billing                             int64
Monthly Charges                             float64
Total Charges                               float64
CLTV                                        float64
Internet Service_DSL                           bool
Internet Service_Fiber optic                   bool
Internet Ser

In [75]:

ada = ADASYN(random_state=130)
x1_resampled, y1_resampled = ada.fit_resample(x1_train, y1_train)
rf = BalancedRandomForestClassifier(random_state=42,n_estimators= 700, max_depth=25, criterion='log_loss', min_samples_split=15, min_samples_leaf=3, max_features='log2', bootstrap=False)
rf.fit(x1_resampled, y1_resampled)

# Predict on the test set
y1_pred = rf.predict(x1_test)

# Evaluate the model
print(confusion_matrix(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

[[861 151]
 [136 259]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      1012
           1       0.63      0.66      0.64       395

    accuracy                           0.80      1407
   macro avg       0.75      0.75      0.75      1407
weighted avg       0.80      0.80      0.80      1407



In [70]:
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(x1_resampled, y1_resampled)


# Predict on the test set
y1_pred = rf.predict(x1_test)

# Evaluate the model
print(confusion_matrix(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

[[876 136]
 [156 239]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      1012
           1       0.64      0.61      0.62       395

    accuracy                           0.79      1407
   macro avg       0.74      0.74      0.74      1407
weighted avg       0.79      0.79      0.79      1407



In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Load the dataset
path = 'Telco_customer_churn.xlsx'
df = pd.read_excel(path)

# Dropping unnecessary columns
df = df.drop(['CustomerID', 'Country', 'Count', 'State', 'Zip Code', 'Lat Long', 'Longitude', 'Latitude', 'City', 'Churn Score', 'Churn Label'], axis=1)

# Replacing specific values
def replace_t(df):
    df.replace({'No internet service': 'No', 'No phone service': 'No'}, inplace=True)
    return df

df = replace_t(df)

# Convert data types
df['Monthly Charges'] = df['Monthly Charges'].astype(float)
df['CLTV'] = df['CLTV'].astype(float)
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')
df['Tenure Months'] = pd.to_numeric(df['Tenure Months'], errors='coerce')
df['Churn Value'] = pd.to_numeric(df['Churn Value'], errors='coerce')

# Remove rows with missing Total Charges
df = df.dropna(subset=['Total Charges'])

# Remove rows where Churn Reason is missing
df2 = df.dropna(subset=['Churn Reason'])

# Splitting the dataset into features and target variables
y1 = df['Churn Value']
x1 = df.drop(columns=['Churn Value', 'Churn Reason'])

# Identifying columns for encoding
num_col = []
LE_col = []
OE_col = []
for col in x1.columns:
    if pd.api.types.is_numeric_dtype(x1[col]):
        num_col.append(col)
    else:
        if x1[col].nunique() == 2:
            LE_col.append(col)
        elif x1[col].nunique() > 2:
            OE_col.append(col)

# One-hot encoding function
def one_hot_encode_columns(df, columns):
    for column in columns:
        one_hot = pd.get_dummies(df[column], prefix=column)
        df = df.drop(column, axis=1)
        df = pd.concat([df, one_hot], axis=1)
    return df

# Label encoding function
LE = LabelEncoder()
def label_encode_columns(df, columns):
    for col in columns:
        df[col] = LE.fit_transform(df[col])
    return df

# Splitting the dataset into training and test sets
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

# Applying one-hot encoding and label encoding
x1_train = one_hot_encode_columns(x1_train, OE_col)
x1_test = one_hot_encode_columns(x1_test, OE_col)
x1_train = label_encode_columns(x1_train, LE_col)
x1_test = label_encode_columns(x1_test, LE_col)

# Apply SMOTETomek sampling
smote_tomek = SMOTETomek(random_state=42)
x1_resampled, y1_resampled = smote_tomek.fit_resample(x1_train, y1_train)

# Train Random Forest
rf = RandomForestClassifier(random_state=42,n_estimators=500, max_depth=20, criterion='entropy', min_samples_split=10, min_samples_leaf=4,max_features='sqrt', bootstrap=False, class_weight='balanced')

rf.fit(x1_resampled, y1_resampled)
y1_pred_rf = rf.predict(x1_test)
rf_results = {
    "confusion_matrix": confusion_matrix(y1_test, y1_pred_rf),
    "classification_report": classification_report(y1_test, y1_pred_rf)
}

# Apply ADASYN sampling
ada = ADASYN(random_state=130)
x1_resampled_ada, y1_resampled_ada = ada.fit_resample(x1_train, y1_train)

# Train Balanced Random Forest
brf = BalancedRandomForestClassifier(random_state=42,n_estimators= 700, max_depth=25, criterion='log_loss', min_samples_split=15, min_samples_leaf=3, max_features='log2', bootstrap=False)
brf.fit(x1_resampled_ada, y1_resampled_ada)
y1_pred_brf = brf.predict(x1_test)
brf_results = {
    "confusion_matrix": confusion_matrix(y1_test, y1_pred_brf),
    "classification_report": classification_report(y1_test, y1_pred_brf)
}

# Train Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(x1_resampled_ada, y1_resampled_ada)
y1_pred_log_reg = log_reg.predict(x1_test)
log_reg_results = {
    "confusion_matrix": confusion_matrix(y1_test, y1_pred_log_reg),
    "classification_report": classification_report(y1_test, y1_pred_log_reg)
}

print("Random Forest Results")
print(rf_results["confusion_matrix"])
print(rf_results["classification_report"])

print("Balanced Random Forest Results")
print(brf_results["confusion_matrix"])
print(brf_results["classification_report"])

print("Logistic Regression Results")
print(log_reg_results["confusion_matrix"])
print(log_reg_results["classification_report"])


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

Random Forest Results
[[862 150]
 [138 257]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      1012
           1       0.63      0.65      0.64       395

    accuracy                           0.80      1407
   macro avg       0.75      0.75      0.75      1407
weighted avg       0.80      0.80      0.80      1407

Balanced Random Forest Results
[[861 151]
 [136 259]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      1012
           1       0.63      0.66      0.64       395

    accuracy                           0.80      1407
   macro avg       0.75      0.75      0.75      1407
weighted avg       0.80      0.80      0.80      1407

Logistic Regression Results
[[852 160]
 [127 268]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86      1012
           1       0.63      0.68      0.65       395

    accuracy                    